## How To: Calculating spectral vegetation indices with EMIT data

This notebook provides examples demonstrating how to create maps of spectral vegetation indices using EMIT images. To explore the utility of high spectral resolution data for calculating SVIs for Earth Science, we can demonstrate how to calculate commonly-used spectral vegetation indices.  For reference, here is a recent article describing the use of SVIs in Earth Science <br>
[https://www.nature.com/articles/s41597-023-02096-0](https://www.nature.com/articles/s41597-023-02096-0)

### Step 1: Load required Python modules

In [ ]:
import os, sys, fnmatch
import warnings
import csv
from osgeo import gdal
import numpy as np
import math
import rasterio as rio
import xarray as xr
import holoviews as hv
import hvplot.xarray
import netCDF4 as nc

sys.path.append(os.path.join(os.path.expanduser("~"),"HYR-SENSE","tools","functions"))
from spectral_index import *

# This will ignore some warnings caused by holoviews
warnings.simplefilter('ignore') 

### Step 2: Define input file path and load example data

In this example we will use the example data downloaded in the other "How To" example **"Download_data_granules_basic_example"**

In [ ]:
### Example image to use
workflow = "examples"
platform = "emit"
source_file_path = os.path.join(os.path.expanduser("~"),"HYR-SENSE","data",workflow,platform)
granules = fnmatch.filter(os.listdir(source_file_path), '*ortho.nc')
granules

In [ ]:
## Load the selected image to memory - we will use the scene ID EMIT_L2A_RFL_001_20230729T205630_2321014_019 in this example
img_file = 'EMIT_L2A_RFL_001_20230729T205630_2321014_019_ortho.nc'
img_file_dat = os.path.join(source_file_path,img_file)
ds_geo = xr.open_dataset(img_file_dat)

### Step 3: Calculate spectral vegetation indices (SVIs)

Below we will walk through how you calculate SVIs remote sensing data, and EMIT hyperspectral imagery in general, why you might select specific band combinations based on what you are studying, and a few different ways you can calculate SVIs using EMIT data, either in-line, defining functions, or using pre-built functions found in the provided [spectral_index.py](https://github.com/CU-ESIIL/HYR-SENSE/blob/main/tools/functions/spectral_index.py) file

First, let's take a moment to consider again what remote sensing imagery like those provided by EMIT can tell us about the underlying vegetation. The internal structure and biochemistry of leaves (A) within a canopy control the optical signatures observed by remote sensing instrumentation (B). The amount of incident radiation that is reflected by, transmitted through, or absorbed by leaves within a canopy is regulated by these structural and biochemical properties of leaves. For example, leaf properties such as a thick cuticle layer, high wax, and/or a large amount of leaf hairs can significantly influence the amount of first-surface reflectance (that is the reflected light directly off the outer leaf layer that does not interact with the leaf interior), causing less solar radiation to penetrate into the leaf. The thickness of the mesophyll layer associated with other properties, such as thicker leaves, can cause higher degree of internal leaf scattering, less transmittance through the leaf, and higher absorption in some wavelengths. Importantly, the diffuse reflectance out of the leaf is that modified by internal leaf properties and contains useful for mapping functional traits (B). High spectral resolution measurements of leaves and plant canopies enable the indirect, non-contact measurement of key structural and chemical absorption features that are associated with the physiological and biochemical properties of plants (B)


**A)** ![leaf_anatomy.jpg](../../images/leaf_anatomy_figure.jpg) **B)** ![spectral_signatures.jpg](../../images/spectral_signatures.jpg)

We can make use of all of the information contained within the emergent spectral signatures provided by vegetation.  We can do this by directly utilizing the spectral profiles of a leaf or an EMIT pixel, or we can instead target specific wavebands provided by data like EMIT to calculate a spectral vegetation index (SVI). SVIs range widely in the wavelengths/bands, structure, and applications. This article provides some background information ([https://www.nature.com/articles/s41597-023-02096-0](https://www.nature.com/articles/s41597-023-02096-0)) but you can find comprehensive list of SVIs [here](https://www.indexdatabase.de/)


For more information, you can review these select articles and resources that discuss how leaf and canopy structure, leaf chemical properties, and stress can alter the spectral signatures we see with remote sensing data like those provided by EMIT and how SVIs provide us a way to easily probe the properties of vegetation remotely.

[Sources of variability in canopy reflectance and the convergent properties of plants](https://doi.org/10.1111/j.1469-8137.2010.03536.x)

[Retrieval of foliar information about plant pigment systems from high resolution spectroscopy](https://doi.org/10.1016/j.rse.2008.10.019)

[Scaling Functional Traits from Leaves to Canopies](https://link.springer.com/chapter/10.1007/978-3-030-33157-3_3)


**Calculate the Normalized Difference Vegetation Index (NDVI)**

Here we calculate a simple yet powerful index, the normalized difference vegetation index (NDVI). NDVI has been used for over 40 years to study changes on the Earth's surface, specifically related to vegetation, stress, and agriculture. For more information on NDVI, you can explore this article from NASA: [https://earthobservatory.nasa.gov/features/MeasuringVegetation/measuring_vegetation_1.php](https://earthobservatory.nasa.gov/features/MeasuringVegetation/measuring_vegetation_1.php)

To calculate NDVI, we need to select which bands we want to include in the calculation. In general, NDVI is defined using a red and near-infrared band, so lets use a band centered at 650nm and another at 850nm, both squarely within the red and NIR wavelength range as shown in the example spectral graph above

The basic structure of the NDVI is: NDVI = (NIR−Red)/(NIR + Red)

In [ ]:
### To calculate NDVI, starting with an EMIT xarray image we can select out the two bands using the example below

refl650 = ds_geo.sel(wavelengths=650, method='nearest')
refl850 = ds_geo.sel(wavelengths=850, method='nearest')
ndvi = (refl850-refl650)/(refl850+refl650)

# Note: The 'nearest' option allows you to define a specific wavelength you want to use and it will use the closest EMIT band to that wavelength

In [ ]:
### Now lets view the resulting NDVI image stretech to display between 0-1
ndvi.hvplot.image(cmap='viridis', geo=True, tiles='ESRI', aspect = 'equal', frame_width=720, clim=(0,1)).opts(title="NDVI Image")

### Using a function to calculate NDVI

We can also create a custom function to calculate different normalized-difference vegetation indices, like NDVI.  We have provided one example here: 

In [ ]:
def normalized_diff_demo(input_xarray = ds_geo, band1=650, band2=850):
    """
    This function takes an input orthorectified EMIT xarray image and calculates a normalized-difference spectral index 
    image based on the selected bands. The assumption is the input image is an EMIT image in xarray format 
    prepared using the emit_xarray() function from emit_tools.

    The function provided in the traditional two-band normalized difference index format, i.e.  NDI = Band2-Band1 / Band2+Band1

    Function parameters:
    param: input_xarray is the input EMIT xarray image currently located in memory; at present this will not accept 
    the full path to a file that hasn't been loaded into memory yet (TODO)
    
    param: band1 the EMIT band number to use for band 1
    
    param: band2 the EMIT band number to use for band 2
    """
    
    reflb1 = input_xarray.sel(wavelengths=band1, method='nearest')
    reflb2 = input_xarray.sel(wavelengths=band2, method='nearest')
    ndiff_image = (reflb2-reflb1)/(reflb2+reflb1)
    
    return(ndiff_image)

In [ ]:
ndvi2 = normalized_diff_demo(input_xarray = ds_geo, band1=650, band2=850)

In [ ]:
ndvi2.hvplot.image(cmap='viridis', geo=True, tiles='ESRI', aspect = 'equal', frame_width=720, clim=(0,1)).opts(title="NDVI Image")

We can also use the stored functions in the HYR-SENSE repository. These functions can be found in the spectral_index.py file https://github.com/CU-ESIIL/HYR-SENSE/blob/main/tools/functions/spectral_index.py


In [ ]:
ndvi3 = normalized_diff(input_xarray = ds_geo, band1=650, band2=850)

In [ ]:
ndvi3.hvplot.image(cmap='viridis', geo=True, tiles='ESRI', aspect = 'equal', frame_width=720, clim=(0,1)).opts(title="NDVI Image")

### Calculate the Normalized-Difference Water Index (NDWI)

We can also calculate a range of other useful indices using the same basic normalized difference vegetation index approach. For example, if we want to investigate the variation in plant / surface water content we can use an SVI such as the Normalized-Difference Water Index [(NDWI)](https://en.wikipedia.org/wiki/Normalized_difference_water_index). Like NDVI, the NDWI uses two different bands to enhance the detection of a plant property, in this case water content, and does that by using a band centered in the NIR and another band centered in the shortwave infrared that is sensitive to plant water content using the following equation: NDWI = (NIR – SWIR)/(NIR + SWIR).

Because it has the same basic structure as NDVI we can still use the same **normalized_diff()** function but instead replace the default bands with 2200nm and 864nm

In [ ]:
ndwi = normalized_diff(input_xarray = ds_geo, band1=2200, band2=864)

In [ ]:
ndwi.hvplot.image(cmap='viridis', geo=True, tiles='ESRI', aspect = 'equal', frame_width=720, clim=(0,1)).opts(title="NDWI Image")

### Calculate red-edge NDVI



In [ ]:
rendvi = normalized_diff(input_xarray = ds_geo, band1=705, band2=750)

In [ ]:
rendvi.hvplot.image(cmap='viridis', geo=True, tiles='ESRI', aspect = 'equal', frame_width=720, clim=(0,1)).opts(title="reNDVI Image")

### Calculate the Photochemical Reflectance Index

In [ ]:
pri_image = pri(input_xarray = ds_geo, band1=531, band2=570, scaled=True)

In [ ]:
pri_image.hvplot.image(cmap='viridis', geo=True, tiles='ESRI', aspect = 'equal', frame_width=720, clim=(0.44,0.48)).opts(title="PRI Image")

### Calculate a simple ratio index

Let's now calculate another common set of indices, called "simple ratio" indices. In this case, we will calculate what is known as the "Water Band Index" (WBI).

In [ ]:
wbi = simple_ratio(input_xarray = ds_geo, band1=900, band2=970)

In [ ]:
wbi.hvplot.image(cmap='viridis', geo=True, tiles='ESRI', aspect = 'equal', frame_width=720, clim=(0.8,1.2)).opts(title="WBI Image")